## This notebook aims to test approach of simple expression classification training of frozen backbone model: see the results of training 

In [1]:
import re
import torchvision
from torchvision import datasets, models, transforms
from torchvision.datasets.folder import default_loader
import math
import collections
from functools import partial
import random
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch import nn
from torch.nn import functional as F
import copy
from torch.utils import model_zoo
from torch.nn import Sequential, BatchNorm1d, BatchNorm2d, Dropout, Module, Linear
import yaml
import argparse
import os
import time
import sys
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(112),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(120),
        transforms.CenterCrop(112),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
from Datasets.MyDataset_expression_OZON import MyDataset_expression_OZON
    
annotations_path_ozon = '/storage_labs/3030/BelyakovM/Face_attributes/ds/db_BuevichP/emochon/OZON_expressions_dataset/train_7expressions_annotation.txt'
image_datasets_ozon = {x: MyDataset_expression_OZON(annotations_path_ozon,x,{'neutral':0, 'happy':1, 'sad':2, 'surprise':3, 'anger':4, 'disgust':5,'fear':6},
                                          data_transforms[x])
                  for x in ['train', 'val']}

In [4]:
from Datasets.MyDataset_expression_AffectPartly import MyDataset_expression_AffectPartly
    
annotations_path = '/storage_labs/3030/BelyakovM/Face_attributes/ds/db_BuevichP/emochon/AffecetNet_partly/original/annotation_'
image_datasets_AffectPartly = {x: MyDataset_expression_AffectPartly(annotations_path,x,{'neutral':0, 'happiness':1, 'sadness':2, 'surprise':3, 'anger':4, 'disgust':5,'fear':6},
                                          data_transforms[x])
                  for x in ['train', 'val']}

## We create training set by concatenating 2 datasets: OZON and AffectNet

In [5]:
image_datasets_AffectPartly_OZON = {'train': torch.utils.data.ConcatDataset([image_datasets_AffectPartly['train'],image_datasets_ozon['train']]),
                                    'val_AffectPartly': image_datasets_AffectPartly['val'],'val_ozon':image_datasets_ozon['val']}
dataloaders_AffectPartly_OZON = {x: torch.utils.data.DataLoader(image_datasets_AffectPartly_OZON[x], batch_size=63,
                                             shuffle=True, num_workers=4) for x in ['train', 'val_AffectPartly','val_ozon']}

dataset_sizes_AffectPartly_OZON = {x: image_datasets_AffectPartly_OZON[x].__len__() for x in ['train', 'val_AffectPartly','val_ozon']}

In [6]:
def train_model(model, criterion, optimizer, scheduler,dataloaders,dataset_sizes, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val_AffectPartly','val_ozon']:
            model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val_ozon' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
class only_expressions_head(nn.Module):
    """
    Creates a MTL model with the encoder from "model_backbone" 
    """
    def __init__(self, model_backbone):
        super(only_expressions_head,self).__init__()
        self.encoder = model_backbone       #fastai function that creates an encoder given an architecture
        self.expression_head = nn.Linear(in_features=1000, out_features=7, bias=True)

    def forward(self,x):

        x = self.encoder(x)
        emotions = self.expression_head(x)

        return emotions


proxyless_frozen_trained_only_expressions = torch.hub.load('mit-han-lab/ProxylessNAS', "proxyless_cpu" , pretrained=True)
proxyless_frozen_trained_only_expressions = only_expressions_head(proxyless_frozen_trained_only_expressions)
proxyless_frozen_trained_only_expressions = proxyless_frozen_trained_only_expressions.to(device)

Using cache found in /root/.cache/torch/hub/mit-han-lab_ProxylessNAS_master


In [9]:
# Freezing backbone params
for module in proxyless_frozen_trained_only_expressions.encoder.modules():
    if isinstance(module,nn.modules.BatchNorm1d):
        module.eval()
    if isinstance(module,nn.modules.BatchNorm2d):
        module.eval()
    if isinstance(module,nn.modules.BatchNorm3d):
        module.eval()
for i in proxyless_frozen_trained_only_expressions.parameters():
    i.requires_grad = False
for param in proxyless_frozen_trained_only_expressions.expression_head.parameters():
    param.requires_grad = True

for name,param in proxyless_frozen_trained_only_expressions.named_parameters():
    if param.requires_grad:
        print(name)

expression_head.weight
expression_head.bias


In [10]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
criterion = nn.CrossEntropyLoss()


optimizer_proxyless_frozen_trained_only_expressions = optim.SGD(proxyless_frozen_trained_only_expressions.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_proxyless_frozen_trained_only_expressions, step_size=7, gamma=0.1)

In [11]:

proxyless_frozen_trained_only_expressions = train_model(proxyless_frozen_trained_only_expressions, criterion,optimizer_proxyless_frozen_trained_only_expressions, exp_lr_scheduler,dataloaders_AffectPartly_OZON,dataset_sizes_AffectPartly_OZON,
                       num_epochs=30)

Epoch 0/29
----------
train Loss: 1.8948 Acc: 0.2890
val_AffectPartly Loss: 1.8747 Acc: 0.2931
val_ozon Loss: 1.7962 Acc: 0.3266

Epoch 1/29
----------
train Loss: 1.8608 Acc: 0.3010
val_AffectPartly Loss: 1.8122 Acc: 0.3117
val_ozon Loss: 1.7819 Acc: 0.3226

Epoch 2/29
----------
train Loss: 1.8621 Acc: 0.3020
val_AffectPartly Loss: 1.8551 Acc: 0.2951
val_ozon Loss: 1.7698 Acc: 0.3310

Epoch 3/29
----------
train Loss: 1.8521 Acc: 0.3053
val_AffectPartly Loss: 1.9643 Acc: 0.2926
val_ozon Loss: 1.8022 Acc: 0.3334

Epoch 4/29
----------
train Loss: 1.8429 Acc: 0.3090
val_AffectPartly Loss: 1.8195 Acc: 0.3066
val_ozon Loss: 1.7563 Acc: 0.3332

Epoch 5/29
----------
train Loss: 1.8546 Acc: 0.3037
val_AffectPartly Loss: 1.9456 Acc: 0.3020
val_ozon Loss: 1.8617 Acc: 0.3377

Epoch 6/29
----------
train Loss: 1.8399 Acc: 0.3089
val_AffectPartly Loss: 1.8148 Acc: 0.3229
val_ozon Loss: 1.7441 Acc: 0.3410

Epoch 7/29
----------
train Loss: 1.6900 Acc: 0.3508
val_AffectPartly Loss: 1.6982 Acc: 0.

In [12]:
torch.save(proxyless_frozen_trained_only_expressions.state_dict(), '/storage_labs/3030/BelyakovM/Face_attributes/Saved_models/proxyless_frozen_backbone_trained_only_expressions.pth')